# 📈 Le Mouvement Brownien - Fondation des Modèles Financiers

## Introduction

Le mouvement Brownien est la pierre angulaire de la finance mathématique moderne. Il décrit le mouvement aléatoire continu d'une particule (ou d'un prix) au fil du temps.

### 🎯 Objectifs de ce notebook
1. Comprendre intuitivement le mouvement Brownien
2. Visualiser et simuler des trajectoires browniennes
3. Découvrir les mouvements Browniens corrélés (clé pour Heston !)
4. Introduire les équations différentielles stochastiques (EDS)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Pour les beaux affichages mathématiques
from IPython.display import display, Math, Latex

## 1. Qu'est-ce que le Mouvement Brownien ?

### 📚 Histoire
Découvert par le botaniste Robert Brown en 1827 en observant des grains de pollen dans l'eau. Albert Einstein l'a formalisé mathématiquement en 1905.

### 💡 Intuition
Imaginez une particule flottant dans l'eau :
- Elle est constamment "poussée" par des molécules d'eau
- Chaque poussée est aléatoire et indépendante
- Le mouvement résultant est chaotique mais suit des lois mathématiques précises

### 📐 Définition mathématique

Un processus stochastique $W_t$ est un mouvement Brownien standard si :

1. $W_0 = 0$ (commence à zéro)
2. Les incréments sont indépendants : $W_t - W_s$ indépendant de $W_s$ pour $t > s$
3. Les incréments suivent une loi normale : $W_t - W_s \sim \mathcal{N}(0, t-s)$
4. Les trajectoires sont continues

### 🔑 Propriété clé
Sur un intervalle de temps $\Delta t$, l'incrément est :
$$\Delta W = W_{t+\Delta t} - W_t \sim \mathcal{N}(0, \Delta t)$$

Ou de manière équivalente : $\Delta W = \sqrt{\Delta t} \times Z$ où $Z \sim \mathcal{N}(0, 1)$

In [ ]:
# Simulation d'un mouvement Brownien standard
def simuler_brownien(T=1.0, N=1000, n_paths=1):
    """
    Simule des trajectoires de mouvement Brownien standard
    
    Paramètres:
    - T : horizon de temps
    - N : nombre de pas de temps
    - n_paths : nombre de trajectoires
    
    Retourne:
    - t : vecteur temps
    - W : matrice des trajectoires (N+1 x n_paths)
    """
    dt = T / N  # pas de temps
    t = np.linspace(0, T, N+1)
    
    # Générer les incréments browniens
    # dW ~ N(0, dt)
    dW = np.random.normal(0, np.sqrt(dt), (N, n_paths))
    
    # Construire le mouvement Brownien par somme cumulative
    W = np.zeros((N+1, n_paths))
    W[1:] = np.cumsum(dW, axis=0)
    
    return t, W

# Simulons 10 trajectoires
np.random.seed(42)
t, W = simuler_brownien(T=1.0, N=1000, n_paths=10)

# Visualisation
plt.figure(figsize=(16, 8))

for i in range(10):
    plt.plot(t, W[:, i], alpha=0.7, linewidth=1.5)

plt.axhline(y=0, color='black', linestyle='--', linewidth=2, alpha=0.5)
plt.title('10 Trajectoires de Mouvement Brownien Standard', fontsize=16, fontweight='bold')
plt.xlabel('Temps t', fontsize=12)
plt.ylabel('W(t)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("🔍 Observations :")
print("  • Toutes les trajectoires commencent à 0")
print("  • Elles sont continues (pas de sauts)")
print("  • Elles sont très irrégulières et imprévisibles")
print("  • L'écart par rapport à 0 augmente avec le temps")

## 2. Propriétés du Mouvement Brownien

### 📊 Vérification empirique des propriétés

In [ ]:
# Générons beaucoup de trajectoires pour analyser les propriétés
np.random.seed(123)
t, W_large = simuler_brownien(T=1.0, N=1000, n_paths=10000)

# Analysons W(t) à différents instants
temps_analyse = [0.25, 0.5, 0.75, 1.0]
indices = [int(T * len(t)) for T in temps_analyse]

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.ravel()

for i, (temps, idx) in enumerate(zip(temps_analyse, indices)):
    W_t = W_large[idx, :]
    
    # Histogramme
    axes[i].hist(W_t, bins=50, density=True, alpha=0.7, 
                 color='skyblue', edgecolor='black')
    
    # Courbe théorique N(0, t)
    x = np.linspace(W_t.min(), W_t.max(), 100)
    theoretical = stats.norm.pdf(x, 0, np.sqrt(temps))
    axes[i].plot(x, theoretical, 'r-', linewidth=3, 
                label=f'N(0, {temps:.2f})')
    
    axes[i].set_title(f'Distribution de W({temps}) - 10,000 trajectoires', 
                     fontsize=12, fontweight='bold')
    axes[i].set_xlabel('Valeur')
    axes[i].set_ylabel('Densité')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)
    
    # Afficher statistiques
    mean_empirique = np.mean(W_t)
    std_empirique = np.std(W_t)
    std_theorique = np.sqrt(temps)
    
    stats_text = f"Moyenne: {mean_empirique:.4f} (théo: 0)\n"
    stats_text += f"Écart-type: {std_empirique:.4f} (théo: {std_theorique:.4f})"
    axes[i].text(0.02, 0.98, stats_text, transform=axes[i].transAxes,
                fontsize=9, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print("✅ Propriété vérifiée: W(t) ~ N(0, t)")
print("   Plus le temps augmente, plus la distribution s'élargit !")

### 🧮 Propriétés mathématiques importantes

1. **Espérance** : $\mathbb{E}[W_t] = 0$
2. **Variance** : $\text{Var}[W_t] = t$
3. **Écart-type** : $\sigma(W_t) = \sqrt{t}$ (augmente avec la racine du temps !)
4. **Non-différentiable** : Les trajectoires sont continues mais très irrégulières
5. **Indépendance des incréments** : $W_{t_2} - W_{t_1}$ est indépendant de $W_{t_1}$

## 3. Mouvement Brownien avec Drift et Volatilité

### 💡 Extension : Mouvement Brownien Géométrique

Pour modéliser des prix, on utilise un mouvement Brownien **avec drift (tendance)** et **volatilité** :

$$dX_t = \mu \, dt + \sigma \, dW_t$$

Où :
- $\mu$ = drift (tendance moyenne)
- $\sigma$ = volatilité (amplitude des fluctuations)
- $dW_t$ = incrément brownien

Solution : $X_t = X_0 + \mu t + \sigma W_t$

### 🎯 Pour les prix d'actifs

On utilise plutôt un **mouvement Brownien géométrique** (MBG) :

$$dS_t = \mu S_t \, dt + \sigma S_t \, dW_t$$

Cela garantit que $S_t > 0$ toujours (un prix ne peut pas être négatif !)

In [ ]:
# Comparaison : Brownien standard vs avec drift vs géométrique
def simuler_MBG(S0, mu, sigma, T=1.0, N=1000, n_paths=1):
    """
    Simule un Mouvement Brownien Géométrique
    dS = mu * S * dt + sigma * S * dW
    """
    dt = T / N
    t = np.linspace(0, T, N+1)
    
    # Générer brownien
    dW = np.random.normal(0, np.sqrt(dt), (N, n_paths))
    
    # Solution exacte du MBG (formule d'Ito)
    S = np.zeros((N+1, n_paths))
    S[0] = S0
    
    for i in range(N):
        S[i+1] = S[i] * np.exp((mu - 0.5*sigma**2)*dt + sigma*dW[i])
    
    return t, S

# Paramètres
S0 = 100
mu = 0.1  # 10% de drift annuel
sigma = 0.2  # 20% de volatilité annuelle
T = 1.0
N = 1000

np.random.seed(42)

# Simulations
t, W = simuler_brownien(T, N, 5)
t, S = simuler_MBG(S0, mu, sigma, T, N, 5)

# Visualisation comparative
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(16, 14))

# 1. Brownien standard
for i in range(5):
    ax1.plot(t, W[:, i], alpha=0.7, linewidth=2)
ax1.axhline(y=0, color='black', linestyle='--', linewidth=2)
ax1.set_title('Mouvement Brownien Standard: W(t)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Temps')
ax1.set_ylabel('W(t)')
ax1.grid(True, alpha=0.3)

# 2. Brownien avec drift
X = S0 + mu * np.tile(t.reshape(-1, 1), (1, 5)) + sigma * W
for i in range(5):
    ax2.plot(t, X[:, i], alpha=0.7, linewidth=2)
ax2.plot(t, S0 + mu * t, 'k--', linewidth=2, label=f'Tendance: {mu}t')
ax2.set_title(f'Mouvement Brownien avec Drift: X(t) = {S0} + {mu}t + {sigma}W(t)', 
             fontsize=14, fontweight='bold')
ax2.set_xlabel('Temps')
ax2.set_ylabel('X(t)')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Brownien géométrique (PRIX)
for i in range(5):
    ax3.plot(t, S[:, i], alpha=0.7, linewidth=2)
ax3.plot(t, S0 * np.exp(mu * t), 'k--', linewidth=2, 
         label=f'Tendance exponentielle: {S0}e^({mu}t)')
ax3.axhline(y=S0, color='red', linestyle='--', linewidth=1, alpha=0.5)
ax3.set_title(f'Mouvement Brownien Géométrique (Prix): dS = {mu}S dt + {sigma}S dW', 
             fontsize=14, fontweight='bold')
ax3.set_xlabel('Temps')
ax3.set_ylabel('S(t) - Prix')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Observations importantes:")
print("  1️⃣ Brownien standard: oscille autour de 0, peut être négatif")
print("  2️⃣ Avec drift: tendance linéaire + fluctuations, peut être négatif")
print("  3️⃣ Géométrique: TOUJOURS POSITIF, tendance exponentielle")
print("\n✅ Pour les PRIX, on utilise le Brownien Géométrique (3️⃣)")

## 4. Mouvements Browniens Corrélés 🔗

### 💡 Pourquoi c'est crucial pour Heston ?

Dans le modèle de Heston, nous avons **deux processus aléatoires** :
1. Le prix de l'actif $S_t$
2. La volatilité $v_t$

Ces deux processus sont **corrélés** ! 

### 🎓 Observation empirique (effet de levier)
- Quand le prix baisse → la volatilité monte (marché nerveux)
- Quand le prix monte → la volatilité baisse (marché calme)
- Corrélation **négative** entre prix et volatilité !

### 📐 Comment générer deux Browniens corrélés ?

Soient $W^1_t$ et $W^2_t$ deux browniens avec corrélation $\rho$ :

$$dW^1_t \cdot dW^2_t = \rho \, dt$$

**Méthode de Cholesky** :
1. Générer $Z_1, Z_2$ indépendants $\sim \mathcal{N}(0,1)$
2. Poser : 
   - $dW^1 = \sqrt{dt} \cdot Z_1$
   - $dW^2 = \sqrt{dt} \cdot (\rho Z_1 + \sqrt{1-\rho^2} Z_2)$

In [ ]:
def simuler_browniens_correles(rho, T=1.0, N=1000, n_paths=1):
    """
    Simule deux mouvements Browniens corrélés
    
    Paramètres:
    - rho : coefficient de corrélation [-1, 1]
    - T, N, n_paths : comme avant
    
    Retourne:
    - t, W1, W2 : temps et deux browniens corrélés
    """
    dt = T / N
    t = np.linspace(0, T, N+1)
    
    # Générer des gaussiennes indépendantes
    Z1 = np.random.normal(0, 1, (N, n_paths))
    Z2 = np.random.normal(0, 1, (N, n_paths))
    
    # Construire les browniens corrélés
    dW1 = np.sqrt(dt) * Z1
    dW2 = np.sqrt(dt) * (rho * Z1 + np.sqrt(1 - rho**2) * Z2)
    
    W1 = np.zeros((N+1, n_paths))
    W2 = np.zeros((N+1, n_paths))
    
    W1[1:] = np.cumsum(dW1, axis=0)
    W2[1:] = np.cumsum(dW2, axis=0)
    
    return t, W1, W2

# Testons différentes corrélations
correlations = [-0.9, -0.5, 0, 0.5, 0.9]
np.random.seed(42)

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for i, rho in enumerate(correlations):
    t, W1, W2 = simuler_browniens_correles(rho, T=1.0, N=1000, n_paths=10)
    
    # Tracer les trajectoires
    for j in range(10):
        axes[i].plot(W1[:, j], W2[:, j], alpha=0.5, linewidth=1)
    
    axes[i].axhline(y=0, color='black', linestyle='--', linewidth=0.5, alpha=0.3)
    axes[i].axvline(x=0, color='black', linestyle='--', linewidth=0.5, alpha=0.3)
    axes[i].set_title(f'ρ = {rho}', fontsize=14, fontweight='bold')
    axes[i].set_xlabel('W¹(t)')
    axes[i].set_ylabel('W²(t)')
    axes[i].grid(True, alpha=0.3)
    axes[i].set_aspect('equal')
    
    # Calculer corrélation empirique
    corr_empirique = np.corrcoef(W1[-1, :], W2[-1, :])[0, 1]
    axes[i].text(0.05, 0.95, f'Corr empirique: {corr_empirique:.3f}',
                transform=axes[i].transAxes, fontsize=10,
                verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Supprimer le 6ème subplot vide
fig.delaxes(axes[5])

plt.suptitle('Browniens Corrélés : Trajectoires dans le plan (W¹, W²)', 
             fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("🔍 Interprétation :")
print("  • ρ = -0.9 : Forte corrélation NÉGATIVE (quand W¹↑ alors W²↓)")
print("  • ρ = 0    : INDÉPENDANTS (nuage circulaire)")
print("  • ρ = +0.9 : Forte corrélation POSITIVE (quand W¹↑ alors W²↑)")
print("\n✅ Dans Heston, ρ < 0 typiquement (effet de levier)")

### 📊 Visualisation 3D des browniens corrélés

In [ ]:
# Visualisation 3D pour mieux comprendre
np.random.seed(123)
rho = -0.7  # Corrélation négative typique de Heston
t, W1, W2 = simuler_browniens_correles(rho, T=1.0, N=500, n_paths=5)

fig = plt.figure(figsize=(16, 6))

# Vue 3D
ax1 = fig.add_subplot(121, projection='3d')
for i in range(5):
    ax1.plot(t, W1[:, i], W2[:, i], alpha=0.8, linewidth=2)
ax1.set_xlabel('Temps t', fontsize=11)
ax1.set_ylabel('W¹(t)', fontsize=11)
ax1.set_zlabel('W²(t)', fontsize=11)
ax1.set_title(f'Browniens corrélés en 3D (ρ = {rho})', fontsize=14, fontweight='bold')

# Nuage de points final
ax2 = fig.add_subplot(122)
np.random.seed(123)
t, W1_large, W2_large = simuler_browniens_correles(rho, T=1.0, N=500, n_paths=1000)
ax2.scatter(W1_large[-1, :], W2_large[-1, :], alpha=0.3, s=10)
ax2.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.3)
ax2.axvline(x=0, color='black', linestyle='--', linewidth=1, alpha=0.3)
ax2.set_xlabel('W¹(T)', fontsize=12)
ax2.set_ylabel('W²(T)', fontsize=12)
ax2.set_title(f'Distribution finale à T=1 (1000 trajectoires)', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.set_aspect('equal')

# Calculer et afficher corrélation
corr = np.corrcoef(W1_large[-1, :], W2_large[-1, :])[0, 1]
ax2.text(0.05, 0.95, f'Corrélation: {corr:.3f}\n(théorique: {rho})',
        transform=ax2.transAxes, fontsize=12, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

plt.tight_layout()
plt.show()

## 🎯 Résumé et Connexion avec Heston

### Ce que nous avons appris :

1. **Mouvement Brownien** : processus aléatoire continu, fondation des modèles financiers
2. **Propriétés** : $W_t \sim \mathcal{N}(0, t)$, incréments indépendants
3. **Brownien géométrique** : pour modéliser des prix (toujours positifs)
4. **Browniens corrélés** : méthode de Cholesky, crucial pour Heston

### 🔗 Lien avec le modèle de Heston

Le modèle de Heston s'écrit :

$$\begin{cases}
dS_t = \mu S_t \, dt + \sqrt{v_t} S_t \, dW^1_t \\
dv_t = \kappa(\theta - v_t) \, dt + \sigma_v \sqrt{v_t} \, dW^2_t
\end{cases}$$

Avec $dW^1_t \cdot dW^2_t = \rho \, dt$

Nous avons maintenant tous les outils pour comprendre cette équation :
- ✅ Mouvement Brownien géométrique pour $S_t$
- ✅ Processus stochastique pour $v_t$ (volatilité)
- ✅ Corrélation $\rho$ entre les deux browniens

### 📖 Prochain notebook : Le Modèle de Heston

Dans le prochain notebook, nous verrons :
- La formulation complète du modèle de Heston
- Le processus CIR pour la volatilité
- Comment simuler le modèle complet
- Calibration des paramètres

---

**Continuez vers le notebook 03_Modele_Heston.ipynb** 🚀